In [135]:
import glob
import pandas as pd
import numpy as np

import os
import re

In [136]:
attack = "crownibp"

In [137]:
results_folder = rf"results\{attack}"

In [138]:
algorithms = ["baseline","noise","fgsm","pgd","crownibp"]

In [167]:
# CI , IBS , NegLL
metric = "NegLL"

In [168]:
ascending = False if metric=="CI" else True

 # aggregate all the CI files
os.listdir(results_folder)
excels = []
for folder in os.listdir(results_folder):
    glob_search = os.path.join(results_folder,folder,"*",f"{metric}.xlsx")
    excels.extend(glob.glob(glob_search))

In [169]:
df = pd.DataFrame()
N_datasets = len(excels)
percentage_change = []
print(excels)
for excel_i in excels:

    temp_df = pd.read_excel(excel_i)
    folder_name,dataset_name = excel_i.split("\\")[-3:-1]
    temp_df.columns = ["eps"] + temp_df.columns[1:].to_list()
    col_name = (dataset_name,re.sub("results_","",folder_name))
    df[col_name] = temp_df[f"Robust {metric}"] #temp_df["Non Robust CI"].round(3).astype(str) + " / " + temp_df["Robust CI"].round(3).astype(str)

    col_name = (dataset_name,"baseline")
    df[col_name] = temp_df[f"Non Robust {metric}"]

df.index = temp_df.eps

['results\\crownibp\\results_crownibp\\Aids2\\NegLL.xlsx', 'results\\crownibp\\results_crownibp\\dataDIVAT1\\NegLL.xlsx', 'results\\crownibp\\results_crownibp\\Dialysis\\NegLL.xlsx', 'results\\crownibp\\results_crownibp\\divorce\\NegLL.xlsx', 'results\\crownibp\\results_crownibp\\flchain\\NegLL.xlsx', 'results\\crownibp\\results_crownibp\\Framingham\\NegLL.xlsx', 'results\\crownibp\\results_crownibp\\prostate\\NegLL.xlsx', 'results\\crownibp\\results_crownibp\\TRACE\\NegLL.xlsx', 'results\\crownibp\\results_fgsm\\Aids2\\NegLL.xlsx', 'results\\crownibp\\results_fgsm\\dataDIVAT1\\NegLL.xlsx', 'results\\crownibp\\results_fgsm\\Dialysis\\NegLL.xlsx', 'results\\crownibp\\results_fgsm\\divorce\\NegLL.xlsx', 'results\\crownibp\\results_fgsm\\flchain\\NegLL.xlsx', 'results\\crownibp\\results_fgsm\\Framingham\\NegLL.xlsx', 'results\\crownibp\\results_fgsm\\prostate\\NegLL.xlsx', 'results\\crownibp\\results_fgsm\\TRACE\\NegLL.xlsx', 'results\\crownibp\\results_noise\\Aids2\\NegLL.xlsx', 'results

In [170]:
micolumns = pd.MultiIndex.from_tuples(df.columns)
dfmi = (
    pd.DataFrame(
        df.values,
        index=df.index,
        columns=micolumns,
    )
    .sort_index()
    .sort_index(axis=1)
)

dfmi.sort_values(by="eps",ascending=False,inplace=True)
dfmi = dfmi.reindex(columns=algorithms, level=1)


In [171]:
dfmi[dfmi==''] = np.NaN
dfmi = dfmi.astype(float)
dfmi

Aids2                                                         \
           baseline         noise          fgsm           pgd     crownibp   
eps                                                                          
2.00  867218.625000  1.664442e+09  5.247136e+07  7.151272e+07  1013.211182   
1.00   11408.425781  3.843492e+05  6.768741e+04  6.786554e+04   529.279114   
0.80    4595.645508  6.678125e+04  1.776805e+04  1.668538e+04   522.198975   
0.70    2868.485596  2.779730e+04  8.894230e+03  8.134271e+03   521.521118   
0.60    1798.874268  1.135636e+04  4.351412e+03  3.943289e+03   521.171753   
0.50    1159.978516  4.424974e+03  2.142487e+03  1.916433e+03   520.945190   
0.10     515.993896  5.196252e+02  5.187775e+02  5.174912e+02   521.655640   
0.07     516.228821  5.188468e+02  5.188582e+02  5.182183e+02   521.751038   
0.05     516.784729  5.191172e+02  5.191102e+02  5.187396e+02   521.819092   
0.00     518.514893  5.208565e+02  5.195742e+02  5.196279e+02   521.989746   

          Dialysis                                                          \
          baseline         noise          fgsm           pgd      crownibp   
eps                                                                          
2.00  4.559872e+16           NaN           NaN           NaN  29887.589844   
1.00  9.240066e+08  6.920299e+34  5.686380e+30  1.546565e+20   2584.852295   
0.80  3.504494e+07  2.610555e+27  2.242915e+24  1.051318e+16   1959.104614   
0.70  7.014061e+06  5.569553e+23  1.570649e+21  9.310418e+13   1776.846558   
0.60  1.445596e+06  1.300917e+20  1.159009e+18  8.956114e+11   1658.694092   
0.50  3.095362e+05  3.559136e+16  8.869410e+14  9.324310e+09   1591.970093   
0.10  1.812542e+03  5.482956e+03  3.559089e+03  1.754042e+03   1523.746338   
0.07  1.644429e+03  2.364513e+03  1.881024e+03  1.558638e+03   1523.031616   
0.05  1.579474e+03  1.844645e+03  1.626077e+03  1.523623e+03   1522.824341   
0.00  1.516727e+03  1.567583e+03  1.529369e+03  1.512401e+03   1523.203857   

      ...       flchain                                            \
      ...      baseline         noise          fgsm           pgd   
eps   ...                                                           
2.00  ...           NaN           NaN           NaN           NaN   
1.00  ...  3.350622e+23  2.180088e+35  1.776159e+22  2.106990e+16   
0.80  ...  2.564323e+18  7.580132e+26  1.907994e+16  4.440591e+11   
0.70  ...  7.372112e+15  4.678509e+22  2.164631e+13  4.052318e+09   
0.60  ...  2.230515e+13  3.480280e+18  3.860451e+10  9.879168e+07   
0.50  ...  6.824825e+10  2.650496e+14  1.937640e+08  1.986687e+06   
0.10  ...  2.033814e+03  1.963553e+03  1.165439e+03  1.105320e+03   
0.07  ...  1.443374e+03  1.371002e+03  1.112564e+03  1.086571e+03   
0.05  ...  1.250709e+03  1.217649e+03  1.097379e+03  1.080582e+03   
0.00  ...  1.086299e+03  1.106499e+03  1.084137e+03  1.075451e+03   

                        prostate                                            \
          crownibp      baseline         noise          fgsm           pgd   
eps                                                                          
2.00  1.400884e+12  1.238172e+11           NaN  9.300456e+29  2.259180e+26   
1.00  6.307257e+03  9.744662e+05  7.003906e+20  4.081706e+14  8.697574e+12   
0.80  1.425336e+03  1.060174e+05  3.745058e+16  4.231508e+11  2.003527e+10   
0.70  1.194910e+03  3.580381e+04  2.847128e+14  1.464802e+10  1.014711e+09   
0.60  1.115522e+03  1.209254e+04  2.283784e+12  5.074684e+08  5.220634e+07   
0.50  1.089056e+03  4.251688e+03  2.025098e+10  1.888130e+07  2.761720e+06   
0.10  1.069681e+03  3.696384e+02  6.656370e+04  1.510415e+03  5.266254e+02   
0.07  1.069362e+03  3.571730e+02  4.035701e+04  1.167495e+03  4.640999e+02   
0.05  1.069202e+03  3.513427e+02  2.941201e+04  1.026740e+03  4.371343e+02   
0.00  1.068962e+03  3.435633e+02  1.371720e+04  7.833487e+02  3.984485e+02   

                    
          crownibp  
eps                 
2.00  2.

In [172]:
# dfmi.T.groupby(axis=0,level=0).rank(axis=0,na_option='bottom',method="average",ascending=ascending)

In [173]:
best_per_dataset_rank = dfmi.groupby(level=0,axis=1).rank(axis=0,na_option='bottom',method="average",ascending=ascending)
best_per_dataset_rank

C:\Users\lpott\AppData\Local\Temp\ipykernel_22812\996602392.py:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  best_per_dataset_rank = dfmi.groupby(level=0,axis=1).rank(axis=0,na_option='bottom',method="average",ascending=ascending)
C:\Users\lpott\AppData\Local\Temp\ipykernel_22812\996602392.py:1: FutureWarning: The 'axis' keyword in DataFrameGroupBy.rank is deprecated and will be removed in a future version. Call without passing 'axis' instead.
  best_per_dataset_rank = dfmi.groupby(level=0,axis=1).rank(axis=0,na_option='bottom',method="average",ascending=ascending)


Aids2                          Dialysis                           ...  \
     baseline noise fgsm  pgd crownibp baseline noise fgsm  pgd crownibp  ...   
eps                                                                       ...   
2.00      2.0   5.0  3.0  4.0      1.0      2.0   4.0  4.0  4.0      1.0  ...   
1.00      2.0   5.0  3.0  4.0      1.0      2.0   5.0  4.0  3.0      1.0  ...   
0.80      2.0   5.0  4.0  3.0      1.0      2.0   5.0  4.0  3.0      1.0  ...   
0.70      2.0   5.0  4.0  3.0      1.0      2.0   5.0  4.0  3.0      1.0  ...   
0.60      2.0   5.0  4.0  3.0      1.0      2.0   5.0  4.0  3.0      1.0  ...   
0.50      2.0   5.0  4.0  3.0      1.0      2.0   5.0  4.0  3.0      1.0  ...   
0.10      1.0   4.0  3.0  2.0      5.0      3.0   5.0  4.0  2.0      1.0  ...   
0.07      1.0   3.0  4.0  2.0      5.0      3.0   5.0  4.0  2.0      1.0  ...   
0.05      1.0   4.0  3.0  2.0      5.0      3.0   5.0  4.0  2.0      1.0  ...   
0.00      1.0   4.0  2.0  3.0      5.0      2.0   5.0  4.0  1.0      3.0  ...   

      flchain                          prostate                           
     baseline noise fgsm  pgd crownibp baseline noise fgsm  pgd crownibp  
eps                                                                       
2.00      3.5   3.5  3.5  3.5      1.0      2.0   5.0  4.0  3.0      1.0  
1.00      4.0   5.0  3.0  2.0      1.0      2.0   5.0  4.0  3.0      1.0  
0.80      4.0   5.0  3.0  2.0      1.0      2.0   5.0  4.0  3.0      1.0  
0.70      4.0   5.0  3.0  2.0      1.0      2.0   5.0  4.0  3.0      1.0  
0.60      4.0   5.0  3.0  2.0      1.0      2.0   5.0  4.0  3.0      1.0  
0.50      4.0   5.0  3.0  2.0      1.0      2.0   5.0  4.0  3.0      1.0  
0.10      5.0   4.0  3.0  2.0      1.0      2.0   5.0  4.0  3.0      1.0  
0.07      5.0   4.0  3.0  2.0      1.0      2.0   5.0  4.0  3.0      1.0  
0.05      5.0   4.0  3.0  2.0      1.0      2.0   5.0  4.0  3.0      1.0  
0.00      4.0   5.0  3.0  2.0      1.0      2.0   5.0  4.0  3.0      1.0  

[10 rows x 40 columns]

In [174]:
best_per_dataset_avg_rank = best_per_dataset_rank.stack(level=1).mean(1).unstack(1).sort_values(by="eps",ascending=False)
best_per_dataset_avg_rank

,baseline,noise,fgsm,pgd,crownibp
eps,,,,,
2.00,2.8125,3.5625,4.1875,3.4375,1.000
1.00,2.8750,4.0000,4.1250,3.0000,1.000
0.80,2.8750,4.0000,4.2500,2.8750,1.000
0.70,2.8750,4.0000,4.2500,2.8750,1.000
0.60,2.8750,4.1250,4.1250,2.8750,1.000
0.50,2.8750,4.1250,4.1250,2.8750,1.000
0.10,3.3750,4.5000,3.0000,2.3750,1.750
0.07,3.3750,4.1250,3.1250,2.5000,1.875
0.05,3.2500,4.2500,3.0000,2.5000,2.000


In [175]:
if not ascending:
    best_per_dataset = dfmi.groupby(level=0,axis=1).idxmax(1)
else:
    best_per_dataset = dfmi.groupby(level=0,axis=1).idxmin(1)
best_per_dataset.applymap(lambda x: x[1])

C:\Users\lpott\AppData\Local\Temp\ipykernel_22812\1722459016.py:4: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  best_per_dataset = dfmi.groupby(level=0,axis=1).idxmin(1)
C:\Users\lpott\AppData\Local\Temp\ipykernel_22812\1722459016.py:4: FutureWarning: DataFrameGroupBy.idxmin with axis=1 is deprecated and will be removed in a future version. Operate on the un-grouped DataFrame instead
  best_per_dataset = dfmi.groupby(level=0,axis=1).idxmin(1)
C:\Users\lpott\AppData\Local\Temp\ipykernel_22812\1722459016.py:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  best_per_dataset.applymap(lambda x: x[1])


,Aids2,Dialysis,Framingham,TRACE,dataDIVAT1,divorce,flchain,prostate
eps,,,,,,,,
2.00,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp
1.00,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp
0.80,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp
0.70,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp
0.60,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp
0.50,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp
0.10,baseline,crownibp,crownibp,crownibp,crownibp,fgsm,crownibp,crownibp
0.07,baseline,crownibp,crownibp,crownibp,crownibp,fgsm,crownibp,crownibp
0.05,baseline,crownibp,crownibp,crownibp,baseline,fgsm,crownibp,crownibp


In [176]:
if metric != "NegLL":
     # aggregate all the CI files
    os.listdir(results_folder)
    excels = []
    for folder in os.listdir(results_folder):
        glob_search = os.path.join(results_folder,folder,f"{metric}_all.xlsx")
        excels.extend(glob.glob(glob_search))
        
    df = pd.DataFrame()
    N_datasets = len(excels)
    percentage_change = []
    for excel_i in excels:
    
        temp_df = pd.read_excel(excel_i)
        folder_name = excel_i.split("\\")[-2]
        temp_df.columns = ["eps"] + temp_df.columns[1:].to_list()
        col_name = re.sub("results_","",folder_name)
        df[col_name] = temp_df["%"] #temp_df["Non Robust CI"].round(3).astype(str) + " / " + temp_df["Robust CI"].round(3).astype(str)
    
    df.index = temp_df.eps
    df = df.reindex(columns=algorithms[1:], level=1)

In [177]:
df

,"(Aids2, crownibp)","(Aids2, baseline)","(dataDIVAT1, crownibp)","(dataDIVAT1, baseline)","(Dialysis, crownibp)","(Dialysis, baseline)","(divorce, crownibp)","(divorce, baseline)","(flchain, crownibp)","(flchain, baseline)",...,"(prostate, noise)","(TRACE, noise)","(Aids2, pgd)","(dataDIVAT1, pgd)","(Dialysis, pgd)","(divorce, pgd)","(flchain, pgd)","(Framingham, pgd)","(prostate, pgd)","(TRACE, pgd)"
eps,,,,,,,,,,,,,,,,,,,,,
2.00,1013.211182,867218.625000,20130.279297,3.917180e+06,29887.589844,4.559872e+16,1093.994751,1.627015e+07,1.400884e+12,NaN,...,NaN,2.254761e+18,7.151272e+07,2.058669e+25,NaN,404785.031250,NaN,2.694231e+18,2.259180e+26,5.021622e+18
1.00,529.279114,11408.425781,918.089050,1.433132e+04,2584.852295,9.240066e+08,1051.740601,2.363391e+04,6.307257e+03,3.350622e+23,...,7.003906e+20,9.113339e+08,6.786554e+04,1.845385e+11,1.546565e+20,3071.479004,2.106990e+16,1.189160e+09,8.697574e+12,3.608588e+08
0.80,522.198975,4595.645508,821.152588,4.946042e+03,1959.104614,3.504494e+07,1048.827271,6.934156e+03,1.425336e+03,2.564323e+18,...,3.745058e+16,1.336189e+07,1.668538e+04,3.472836e+08,1.051318e+16,1638.452515,4.440591e+11,1.848761e+07,2.003527e+10,4.441628e+06
0.70,521.521118,2868.485596,800.327820,3.008757e+03,1776.846558,7.014061e+06,1047.909302,4.025990e+03,1.194910e+03,7.372112e+15,...,2.847128e+14,1.743518e+06,8.134271e+03,1.747805e+07,9.310418e+13,1341.789185,4.052318e+09,2.408280e+06,1.014711e+09,5.252624e+05
0.60,521.171753,1798.874268,785.855286,1.914498e+03,1658.694092,1.445596e+06,1047.239380,2.521232e+03,1.115522e+03,2.230515e+13,...,2.283784e+12,2.425280e+05,3.943289e+03,9.390763e+05,8.956114e+11,1175.897095,9.879168e+07,3.242479e+05,5.220634e+07,6.688381e+04
0.50,520.945190,1159.978516,774.627319,1.311128e+03,1591.970093,3.095362e+05,1046.801880,1.788535e+03,1.089056e+03,6.824825e+10,...,2.025098e+10,3.712241e+04,1.916433e+03,6.136595e+04,9.324310e+09,1093.048584,1.986687e+06,4.729048e+04,2.761720e+06,9.688615e+03
0.10,521.655640,515.993896,751.842834,7.562355e+02,1523.746338,1.812542e+03,1045.666992,1.056980e+03,1.069681e+03,2.033814e+03,...,6.656370e+04,5.928043e+02,5.174912e+02,7.574421e+02,1.754042e+03,1045.289673,1.105320e+03,1.514609e+03,5.266254e+02,5.354637e+02
0.07,521.751038,516.228821,751.301270,7.522886e+02,1523.031616,1.644429e+03,1045.636963,1.050426e+03,1.069362e+03,1.443374e+03,...,4.035701e+04,5.640048e+02,5.182183e+02,7.561345e+02,1.558638e+03,1045.443481,1.086571e+03,1.505058e+03,4.640999e+02,5.302774e+02
0.05,521.819092,516.784729,751.017822,7.505762e+02,1522.824341,1.579474e+03,1045.620850,1.047734e+03,1.069202e+03,1.250709e+03,...,2.941201e+04,5.506025e+02,5.187396e+02,7.560538e+02,1.523623e+03,1045.539795,1.080582e+03,1.501635e+03,4.371343e+02,5.279315e+02


In [178]:
excel_name = os.path.join(results_folder,f"{metric}_all.xlsx")
with pd.ExcelWriter(excel_name) as writer:  
    dfmi.applymap(lambda x: np.round(x,3)).to_excel(writer,sheet_name=metric)
    best_per_dataset_rank.to_excel(writer,sheet_name="rank")
    best_per_dataset_avg_rank.to_excel(writer,sheet_name="average_rank")
    best_per_dataset.applymap(lambda x: x[1]).to_excel(writer,sheet_name="best")
    df.to_excel(writer,sheet_name="%")

C:\Users\lpott\AppData\Local\Temp\ipykernel_22812\229516633.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  dfmi.applymap(lambda x: np.round(x,3)).to_excel(writer,sheet_name=metric)
C:\Users\lpott\AppData\Local\Temp\ipykernel_22812\229516633.py:6: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  best_per_dataset.applymap(lambda x: x[1]).to_excel(writer,sheet_name="best")


In [179]:
best_per_dataset_avg_rank.index = ["{:.2f}".format(float(x)) for x in np.round(best_per_dataset_avg_rank.index.tolist(),2)]
best_per_dataset_avg_rank.index.name = r"$\epsilon$"
print(best_per_dataset_avg_rank.applymap(lambda x: str(np.round(x,2))).to_latex(index=True))

\begin{tabular}{llllll}
\toprule
 & baseline & noise & fgsm & pgd & crownibp \\
$\epsilon$ &  &  &  &  &  \\
\midrule
2.00 & 2.81 & 3.56 & 4.19 & 3.44 & 1.0 \\
1.00 & 2.88 & 4.0 & 4.12 & 3.0 & 1.0 \\
0.80 & 2.88 & 4.0 & 4.25 & 2.88 & 1.0 \\
0.70 & 2.88 & 4.0 & 4.25 & 2.88 & 1.0 \\
0.60 & 2.88 & 4.12 & 4.12 & 2.88 & 1.0 \\
0.50 & 2.88 & 4.12 & 4.12 & 2.88 & 1.0 \\
0.10 & 3.38 & 4.5 & 3.0 & 2.38 & 1.75 \\
0.07 & 3.38 & 4.12 & 3.12 & 2.5 & 1.88 \\
0.05 & 3.25 & 4.25 & 3.0 & 2.5 & 2.0 \\
0.00 & 2.75 & 4.62 & 3.12 & 2.38 & 2.12 \\
\bottomrule
\end{tabular}



C:\Users\lpott\AppData\Local\Temp\ipykernel_22812\3581541270.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  print(best_per_dataset_avg_rank.applymap(lambda x: str(np.round(x,2))).to_latex(index=True))


In [180]:
if metric == "NegLL":
    dfmi.index = ["{:.2f}".format(float(x)) for x in np.round(dfmi.index.tolist(),2)]
    dfmi.index.name = r"$\epsilon$"
    print(dfmi.applymap(lambda x: np.round(x,3)).applymap(lambda x: "{:.2e}".format(x)).to_latex(index=True,multicolumn_format="c"))
else:
    dfmi.index = ["{:.2f}".format(float(x)) for x in np.round(dfmi.index.tolist(),2)]
    dfmi.index.name = r"$\epsilon$"
    print(dfmi.applymap(lambda x: np.round(x,3)).applymap(str).to_latex(index=True,multicolumn_format="c"))

\begin{tabular}{lllllllllllllllllllllllllllllllllllllllll}
\toprule
 & \multicolumn{5}{c}{Aids2} & \multicolumn{5}{c}{Dialysis} & \multicolumn{5}{c}{Framingham} & \multicolumn{5}{c}{TRACE} & \multicolumn{5}{c}{dataDIVAT1} & \multicolumn{5}{c}{divorce} & \multicolumn{5}{c}{flchain} & \multicolumn{5}{c}{prostate} \\
 & baseline & noise & fgsm & pgd & crownibp & baseline & noise & fgsm & pgd & crownibp & baseline & noise & fgsm & pgd & crownibp & baseline & noise & fgsm & pgd & crownibp & baseline & noise & fgsm & pgd & crownibp & baseline & noise & fgsm & pgd & crownibp & baseline & noise & fgsm & pgd & crownibp & baseline & noise & fgsm & pgd & crownibp \\
$\epsilon$ &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  \\
\midrule
2.00 & 8.67e+05 & 1.66e+09 & 5.25e+07 & 7.15e+07 & 1.01e+03 & 4.56e+16 & nan & nan & nan & 2.99e+04 & 2.77e+08 & 3.76e+12 & 2.82e+19 & 2.69e+18 & 3.17e+03 & 9.02e+19 & 2.25e+18 & 3.63e+25 & 5.0

C:\Users\lpott\AppData\Local\Temp\ipykernel_22812\718377934.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  print(dfmi.applymap(lambda x: np.round(x,3)).applymap(lambda x: "{:.2e}".format(x)).to_latex(index=True,multicolumn_format="c"))


In [118]:
# pd.set_option('display.max_columns', None)
# print(dfmi)

In [119]:
print(best_per_dataset.applymap(lambda x: x[1]).to_latex(index=False))

\begin{tabular}{llllllll}
\toprule
Aids2 & Dialysis & Framingham & TRACE & dataDIVAT1 & divorce & flchain & prostate \\
\midrule
crownibp & crownibp & crownibp & crownibp & crownibp & crownibp & crownibp & crownibp \\
crownibp & crownibp & crownibp & crownibp & crownibp & crownibp & crownibp & crownibp \\
crownibp & crownibp & crownibp & crownibp & crownibp & crownibp & crownibp & crownibp \\
crownibp & crownibp & crownibp & crownibp & crownibp & crownibp & crownibp & crownibp \\
crownibp & crownibp & crownibp & crownibp & crownibp & crownibp & crownibp & crownibp \\
crownibp & crownibp & crownibp & crownibp & crownibp & crownibp & crownibp & crownibp \\
pgd & crownibp & crownibp & crownibp & crownibp & pgd & crownibp & crownibp \\
pgd & crownibp & crownibp & crownibp & crownibp & pgd & crownibp & crownibp \\
pgd & pgd & crownibp & crownibp & crownibp & pgd & crownibp & crownibp \\
pgd & pgd & crownibp & crownibp & crownibp & noise & crownibp & crownibp \\
\bottomrule
\end{tabular}



C:\Users\lpott\AppData\Local\Temp\ipykernel_22812\3574660769.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  print(best_per_dataset.applymap(lambda x: x[1]).to_latex(index=False))


In [120]:
df.index = ["{:.2f}".format(float(x)) for x in np.round(df.index.tolist(),2)]
df.index.name = r"$\epsilon$"
print(df.applymap(lambda x: str(np.round(x,3))).to_latex(index=True))

\begin{tabular}{lllll}
\toprule
 & noise & fgsm & pgd & crownibp \\
$\epsilon$ &  &  &  &  \\
\midrule
2.00 & 0.05 & 0.042 & 0.04 & -20.616 \\
1.00 & 0.847 & 1.08 & 0.068 & -49.472 \\
0.80 & 2.48 & 2.649 & 0.872 & -60.952 \\
0.70 & 4.37 & 4.598 & 2.205 & -62.723 \\
0.60 & 7.97 & 8.528 & 4.963 & -63.153 \\
0.50 & 13.521 & 15.279 & 8.646 & -61.491 \\
0.10 & 14.154 & -0.208 & -9.155 & -15.937 \\
0.07 & 9.054 & -2.194 & -6.919 & -10.478 \\
0.05 & 6.695 & -1.602 & -4.5 & -7.028 \\
0.00 & 6.037 & 2.581 & 1.085 & -0.674 \\
\bottomrule
\end{tabular}



C:\Users\lpott\AppData\Local\Temp\ipykernel_22812\3740848601.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  print(df.applymap(lambda x: str(np.round(x,3))).to_latex(index=True))


## PDF PICTURES

In [65]:
import matplotlib.pyplot as plt
import os
import glob
import numpy as np
import pandas as pd
import re

In [ ]:
attack = "crownibp"

In [ ]:
results_folder = rf"results\{attack}"

In [ ]:
algorithms = ["baseline","noise","fgsm","pgd","crownibp"]

In [ ]:
# CI , IBS , NegLL
img_name = "population_curves_attacked_test"

 # aggregate all the CI files
os.listdir(results_folder)
excels = []
for folder in os.listdir(results_folder):
    glob_search = os.path.join(results_folder,folder,"*",f"{img_name}.xlsx")
    excels.extend(glob.glob(glob_search))

In [ ]:
metadata = 'eps'

In [ ]:
n_rows = int(len(excels) / (len(algorithms)-1))
n_cols = int(len(algorithms) - 1)

In [ ]:
excels = np.sort(excels)
files = pd.DataFrame(excels.reshape(-1,len(algorithms)-1,order="F"),columns=["crownibp","fgsm","noise","pgd"]).reindex(["noise","fgsm","pgd","crownibp"],axis=1).values

In [ ]:
df_population = pd.read_excel(excels[0])

In [ ]:
df_population

In [ ]:
save_folder = results_folder = os.path.join(fr"C:\Users\lpott\OneDrive\Documents\Northeastern University\Classes\EECE 7398 Verifiable Machine Learning\EECE7398_Final_Project\results\{attack}","perturb_curves.pdf")

In [ ]:
print(save_folder)

In [ ]:
save_folder = results_folder = os.path.join(r"C:\Users\lpott\OneDrive\Documents\Northeastern University\Classes\EECE 7398 Verifiable Machine Learning\EECE7398_Final_Project\results",attack,"perturb_curves.pdf")

fig,axes = plt.subplots(n_rows,n_cols+1,figsize=(30,40),sharey=True)

SMALL_SIZE = 20
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)  

for i in range(n_rows):
    for j in range(n_cols):
        print(i,j)
        fileij = files[i][j]
        algo,dataset = fileij.split("\\")[-3:-1]
        algo = re.sub("results_","",algo)

        df_population = pd.read_excel(fileij)

        t = df_population.pop("t")
        base_models = df_population.iloc[:,:3]
        base_models.columns = [col.split("_")[0] for col in base_models.columns]
        
        df_population= df_population.iloc[:,3:]

        robust_idx = ["robust" in col for col in df_population.columns]
        baseline_idx = ["baseline" in col for col in df_population.columns]
        robust_df = df_population.iloc[:,robust_idx]
        baseline_df = df_population.iloc[:,baseline_idx]
        
        robust_df.columns = ["$\epsilon$={:.2f}".format(eval(col.split("=")[1])) for col in robust_df.columns]
        baseline_df.columns = ["$\epsilon$={:.2f}".format(eval(col.split("=")[1])) for col in baseline_df.columns]
        
        base1 = axes[i][j+1].plot(t,base_models.iloc[:,0],linewidth=3,c="b")
        base2 = axes[i][j+1].plot(t,base_models.iloc[:,1],linewidth=3,c="r")
        base3 = axes[i][j+1].plot(t,base_models.iloc[:,2],linewidth=3,c="k")

  
        robust1 = axes[i][j+1].plot(t,robust_df,'--',linewidth=3)
  
        if j == 0:
            base1 = axes[i][j].plot(t,base_models.iloc[:,0],linewidth=3,c="b")
            base2 = axes[i][j].plot(t,base_models.iloc[:,1],linewidth=3,c="r")
            base3 = axes[i][j].plot(t,base_models.iloc[:,2],linewidth=3,c="k")
            baseline1 = axes[i][j].plot(t,baseline_df,'--',linewidth=3)

            axes[i][j].set_ylabel(f"S(t) {dataset}" ,fontsize=30)
            axes[i][j].set_xlabel("t",fontsize=20)

        axes[i][j+1].set_xlabel("t",fontsize=20)

for ax, col in zip(axes[0], algorithms):
    ax.set_title(col,fontsize=30)


labels = base_models.columns.tolist() + robust_df.columns.tolist()
fig.legend([base1, base2,base3,robust1], labels=labels, 
           loc="upper center",ncols=4,fontsize=30,bbox_to_anchor=(.5,1.06)) 

# axes[0][3].legend(base_models.columns.tolist() + robust_df.columns.tolist(),fontsize=20,ncol=2,loc=1)

# plt.legend(base_models.columns.tolist() + robust_df.columns.tolist(),loc='upper center',ncol=5)
plt.tight_layout()
plt.savefig(save_folder,dpi=1600,bbox_inches="tight")
plt.show()

In [ ]:
files[2:4,:]

In [ ]:
save_folder = results_folder = os.path.join(r"C:\Users\lpott\OneDrive\Documents\Northeastern University\Classes\EECE 7398 Verifiable Machine Learning\EECE7398_Final_Project\results",attack,"perturb_curves_subset.pdf")
file_subset = files[2:4,:]

fig,axes = plt.subplots(2,n_cols+1,figsize=(30,12),sharey=True)

SMALL_SIZE = 20
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)  

for i in range(2):
    for j in range(n_cols):
        print(i,j)
        fileij = file_subset[i][j]
        algo,dataset = fileij.split("\\")[-3:-1]
        if dataset not in ["TRACE","Framingham"]:
            continue
        algo = re.sub("results_","",algo)

        df_population = pd.read_excel(fileij)

        t = df_population.pop("t")
        base_models = df_population.iloc[:,:3]
        base_models.columns = [col.split("_")[0] for col in base_models.columns]
        
        df_population= df_population.iloc[:,3:]

        robust_idx = ["robust" in col for col in df_population.columns]
        baseline_idx = ["baseline" in col for col in df_population.columns]
        robust_df = df_population.iloc[:,robust_idx]
        baseline_df = df_population.iloc[:,baseline_idx]
        
        robust_df.columns = ["$\epsilon$={:.2f}".format(eval(col.split("=")[1])) for col in robust_df.columns]
        baseline_df.columns = ["$\epsilon$={:.2f}".format(eval(col.split("=")[1])) for col in baseline_df.columns]
        
        base1 = axes[i][j+1].plot(t,base_models.iloc[:,0],linewidth=3,c="b")
        base2 = axes[i][j+1].plot(t,base_models.iloc[:,1],linewidth=3,c="r")
        base3 = axes[i][j+1].plot(t,base_models.iloc[:,2],linewidth=3,c="k")

  
        robust1 = axes[i][j+1].plot(t,robust_df,'--',linewidth=3)
  
        if j == 0:
            base1 = axes[i][j].plot(t,base_models.iloc[:,0],linewidth=3,c="b")
            base2 = axes[i][j].plot(t,base_models.iloc[:,1],linewidth=3,c="r")
            base3 = axes[i][j].plot(t,base_models.iloc[:,2],linewidth=3,c="k")
            baseline1 = axes[i][j].plot(t,baseline_df,'--',linewidth=3)

            axes[i][j].set_ylabel(f"S(t) {dataset}" ,fontsize=30)
            axes[i][j].set_xlabel("t",fontsize=20)

        axes[i][j+1].set_xlabel("t",fontsize=20)

for ax, col in zip(axes[0], algorithms):
    ax.set_title(col,fontsize=30)


labels = base_models.columns.tolist() + robust_df.columns.tolist()
fig.legend([base1, base2,base3,robust1], labels=labels, 
           loc="upper center",ncols=6,fontsize=30,bbox_to_anchor=(.5,1.15)) 

# axes[0][3].legend(base_models.columns.tolist() + robust_df.columns.tolist(),fontsize=20,ncol=2,loc=1)

# plt.legend(base_models.columns.tolist() + robust_df.columns.tolist(),loc='upper center',ncol=5)
plt.tight_layout()
plt.savefig(save_folder,dpi=1600,bbox_inches="tight")
plt.show()

### DIST PLOTS

In [ ]:
# CI , IBS , NegLL
results_folder = fr"results\{attack}"
img_name = "curve_distributions_test"

 # aggregate all the CI files
os.listdir(results_folder)
excels = []
for folder in os.listdir(results_folder):
    glob_search = os.path.join(results_folder,folder,"*",f"{img_name}.xlsx")
    excels.extend(glob.glob(glob_search))

In [ ]:
excels = np.sort(excels)
files = pd.DataFrame(excels.reshape(-1,len(algorithms)-1,order="F"),columns=["crownibp","fgsm","noise","pgd"]).reindex(["noise","fgsm","pgd","crownibp"],axis=1).values

In [ ]:
save_folder = results_folder = os.path.join(r"results",attack,"dist_curves.pdf")

In [ ]:
import seaborn as sns

In [ ]:
fig,axes = plt.subplots(n_rows,n_cols+1,figsize=(30,40),sharey=True)

SMALL_SIZE = 20
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)  

for i in range(n_rows):
    for j in range(n_cols):
        print(i,j)
        fileij = files[i][j]
        algo,dataset = fileij.split("\\")[-3:-1]
        algo = re.sub("results_","",algo)

        df_population = pd.read_excel(fileij)

        t = df_population.pop("t")
        base_models = df_population.iloc[:,:3]
        base_models.columns = [col.split("_")[0] for col in base_models.columns]
        
        robust_idx = ["robust" in col for col in df_population.columns]
        baseline_idx = ["baseline" in col for col in df_population.columns]
        robust_df = df_population.iloc[:,robust_idx]
        baseline_df = df_population.iloc[:,baseline_idx]
        
        robust_df.columns = [col.split("_")[1] for col in robust_df.columns]
        baseline_df.columns =[col.split("_")[1] for col in baseline_df.columns]
        
        mu = sns.lineplot(x=t, y=robust_df.iloc[:,0], label='Average S(t)', linewidth=3.0, ax=axes[i][j+1],c='b',legend=False)
        q95 = sns.lineplot(x=t, y=robust_df.iloc[:,1], label='Confidence', linewidth=3.0, ax=axes[i][j+1],c='r',legend=False)
        q05 = sns.lineplot(x=t, y=robust_df.iloc[:,2], label='Confidence', linewidth=3.0, ax=axes[i][j+1],c='r',legend=False)

        line = q05.get_lines()
        axes[i][j+1].fill_between(line[0].get_xdata(), line[1].get_ydata(), line[2].get_ydata(), color='blue', alpha=.3)
        axes[i][j+1].set_xlabel("t",fontsize=20)

        if j == 0:
            mu = sns.lineplot(x=t, y=baseline_df.iloc[:,0], label='Average S(t)', linewidth=3.0, ax=axes[i][j],c='b',legend=False)
            q95 = sns.lineplot(x=t, y=baseline_df.iloc[:,1], label='Confidence', linewidth=3.0, ax=axes[i][j],c='r',legend=False)
            q05 = sns.lineplot(x=t, y=baseline_df.iloc[:,2], label='Confidence', linewidth=3.0, ax=axes[i][j],c='r',legend=False)

            axes[i][j].set_ylabel(f"S(t) {dataset}" ,fontsize=30)
            axes[i][j].set_xlabel("t",fontsize=20)
            line = q05.get_lines()
            axes[i][j].fill_between(line[0].get_xdata(), line[1].get_ydata(), line[2].get_ydata(), color='blue', alpha=.3)
            axes[i][j].set_xlabel("t",fontsize=20)

for ax, col in zip(axes[0], algorithms):
    ax.set_title(col,fontsize=30)


labels = ["S(t)","Credible Interval","$Q_{95},Q_{05}$"]
fig.legend([mu,q95,q05], labels=labels, 
           loc="upper center",ncols=4,fontsize=30,bbox_to_anchor=(.5,1.03)) 

plt.tight_layout()
plt.savefig(save_folder,dpi=1600,bbox_inches="tight")
plt.show()

In [ ]:
save_folder = results_folder = os.path.join(r"results",attack,"dist_curves_subset.pdf")
file_subset = files[2:4,:]

fig,axes = plt.subplots(2,n_cols+1,figsize=(30,12),sharey=True)

SMALL_SIZE = 20
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)  

for i in range(2):
    for j in range(n_cols):
        print(i,j)
        fileij = file_subset[i][j]
        algo,dataset = fileij.split("\\")[-3:-1]
        algo = re.sub("results_","",algo)

        df_population = pd.read_excel(fileij)

        t = df_population.pop("t")
        base_models = df_population.iloc[:,:3]
        base_models.columns = [col.split("_")[0] for col in base_models.columns]
        
        robust_idx = ["robust" in col for col in df_population.columns]
        baseline_idx = ["baseline" in col for col in df_population.columns]
        robust_df = df_population.iloc[:,robust_idx]
        baseline_df = df_population.iloc[:,baseline_idx]
        
        robust_df.columns = [col.split("_")[1] for col in robust_df.columns]
        baseline_df.columns =[col.split("_")[1] for col in baseline_df.columns]
        
        mu = sns.lineplot(x=t, y=robust_df.iloc[:,0], label='Average S(t)', linewidth=3.0, ax=axes[i][j+1],c='b',legend=False)
        q95 = sns.lineplot(x=t, y=robust_df.iloc[:,1], label='Confidence', linewidth=3.0, ax=axes[i][j+1],c='r',legend=False)
        q05 = sns.lineplot(x=t, y=robust_df.iloc[:,2], label='Confidence', linewidth=3.0, ax=axes[i][j+1],c='r',legend=False)
        axes[i][j+1].set_xlabel("t",fontsize=20)

        line = q05.get_lines()
        axes[i][j+1].fill_between(line[0].get_xdata(), line[1].get_ydata(), line[2].get_ydata(), color='blue', alpha=.3)
        if j == 0:
            mu = sns.lineplot(x=t, y=baseline_df.iloc[:,0], label='Average S(t)', linewidth=3.0, ax=axes[i][j],c='b',legend=False)
            q95 = sns.lineplot(x=t, y=baseline_df.iloc[:,1], label='Confidence', linewidth=3.0, ax=axes[i][j],c='r',legend=False)
            q05 = sns.lineplot(x=t, y=baseline_df.iloc[:,2], label='Confidence', linewidth=3.0, ax=axes[i][j],c='r',legend=False)

            axes[i][j].set_ylabel(f"S(t) {dataset}" ,fontsize=30)
            axes[i][j].set_xlabel("t",fontsize=20)
            line = q05.get_lines()
            axes[i][j].fill_between(line[0].get_xdata(), line[1].get_ydata(), line[2].get_ydata(), color='blue', alpha=.3)
            axes[i][j].set_xlabel("t",fontsize=20)

for ax, col in zip(axes[0], algorithms):
    ax.set_title(col,fontsize=30)


labels = ["S(t)","Credible Interval","$Q_{95},Q_{05}$"]
fig.legend([mu,q95,q05], labels=labels, 
           loc="upper center",ncols=4,fontsize=30,bbox_to_anchor=(.5,1.12)) 

plt.tight_layout()
plt.savefig(save_folder,dpi=1600,bbox_inches="tight")
plt.show()